# Enunciado
Hola ingeniero 👋. Bienvenido a tu cuarto desafio! Ya sabes que en [Vault-Tec Corporation](https://fallout.fandom.com/es/wiki/Vault-Tec_Corporation) tenemos sensores encargados de monitorear las condiciones actuales de nuestros refugios. En ocasiones pasadas detectamos que varios de ellos presentaban fallas recurrentes, impidiendo que pudieramos determinar las condiciones de nuestros bóvedas y darle la seguridad esperada a nuestros clientes en un mundo post apocaliptico 🙃. 

Anteriormente, a pedido de nuestro equipo, construimos un pipeline de datos el cual automatiza las actividades relacionadas a soporte y mantenimiento de los distintos sensores presentes en los refugios. 

Sin embargo, luego de haber probado nuestra solucion, nuestro equipo ha detectado una serie de problemas y nos los ha comunicado, en esta ocasion deberemos de solucionarlos.

# Insumos
Tu equipo te ha dado el siguiente codigo el cual deberas modificar para arreglar los problemas detecatados.

**IMPORTANTE:** En la vida real lo comun es que pases mas tiempo leyendo codigo de otros, que escribiendolo. 

In [1]:
from datetime import datetime
from typing import Literal

from pydantic import BaseModel


class Log(BaseModel):
    """Representa un registro (log) generado por un sensor.

    Atributos:
        sensor_id (Literal): Identificador del sensor que generó el registro.
            Valores posibles: "oxy_guard", "thermo_core", "hidra_flow", "radi_shield".
        event_type (Literal): Tipo de evento registrado.
            Valores posibles: "FAILURE_START" (inicio de falla) o "FAILURE_END" (fin de falla).
        timestamp (datetime): Fecha y hora exacta en que ocurrió el evento.
        duration_seconds (int): Duración de la falla en segundos. En caso de ser "FAILURE_START",
            puede ser 0 si aún no ha finalizado la falla.
    """

    sensor_id: Literal["oxy_guard", "thermo_core", "hidra_flow", "radi_shield"]
    event_type: Literal["FAILURE_END", "FAILURE_START"]
    timestamp: datetime
    duration_seconds: int

In [2]:
import json
from datetime import date
from os.path import abspath


class LogExtractor:
    """Clase para extraer y filtrar logs desde un archivo JSON."""

    def __init__(self, file_path: str):
        """
        Inicializa el extractor de logs con la ruta del archivo.

        Args:
            file_path (str): Ruta del archivo JSON que contiene los logs.
        """
        self.file_abspath = abspath(file_path)

    def from_file(self) -> list["Log"]:
        """
        Carga los logs desde el archivo JSON especificado en la inicialización.

        Returns:
            list[Log]: Lista de instancias de Log extraídas del archivo.
        """
        with open(self.file_abspath) as file:
            raw_logs: list[dict] = json.load(file)
        logs: list[Log] = [Log(**log) for log in raw_logs]
        return logs

    @staticmethod
    def fetch_logs_on_date(
        target_date: date, logs: list["Log"], sensor_id: int | None = None
    ) -> list["Log"]:
        """
        Filtra los logs que corresponden a una fecha y, opcionalmente, a un sensor específico.

        Args:
            target_date (date): Fecha objetivo para filtrar los logs.
            logs (list[Log]): Lista de logs disponibles.
            sensor_id (Optional[int], optional): ID del sensor para filtrar.
                Si es None, se incluyen logs de todos los sensores. Defaults to None.

        Returns:
            list[Log]: Lista de logs que cumplen con los filtros aplicados.
        """
        logs_on_date: list[Log] = list(
            filter(
                lambda x: (
                    x.timestamp.date() == target_date
                    and (sensor_id is None or x.sensor_id == sensor_id)
                ),
                logs,
            )
        )
        return logs_on_date

In [3]:
from datetime import date, timedelta


class SensorFailureAnalyzer:
    """Clase para analizar fallos de sensores a partir de registros de eventos.

    Esta clase permite calcular la duración de fallos de sensores,
    la probabilidad de fallo semanal y la probabilidad condicional
    de que un sensor falle dado que otro sensor también falló.
    """

    WEEK_TOTAL_SECONDS: int = 7 * 24 * (60**2)
    """int: Número total de segundos en una semana."""

    def __init__(self, logs: list[Log]):
        """Inicializa el analizador con los registros de logs.

        Args:
            logs (list[Log]): Lista de objetos `Log` que contienen los eventos de los sensores.
        """
        self.logs = logs

    def get_sensor_failure_duration(self, sensor_id: str, searched_date: date) -> float | None:
        """Obtiene la duración total del fallo de un sensor en una fecha específica.

        Busca en los logs los eventos `FAILURE_START` y `FAILURE_END`
        del sensor en la fecha indicada y calcula el tiempo total en segundos.

        Args:
            sensor_id (str): Identificador del sensor a analizar.
            searched_date (date): Fecha para la que se quiere obtener la duración del fallo.

        Returns:
            float | None: Duración del fallo en segundos.
            Devuelve `None` si no hay logs del sensor en esa fecha.
        """
        logs_on_date: list[Log] = LogExtractor.fetch_logs_on_date(
            sensor_id=sensor_id, target_date=searched_date, logs=self.logs
        )

        if not logs_on_date:
            return None

        failure_start_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_START"), 0
        )
        failure_end_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_END"), 0
        )

        return abs(failure_end_time - failure_start_time)

    def get_weekly_sensor_failure_duration(
        self, sensor_id: int, start_date: date, end_date: date | None = None
    ) -> list[dict]:
        """Obtiene la duración diaria de fallos de un sensor durante una semana.

        Args:
            sensor_id (int): Identificador del sensor.
            start_date (date): Fecha de inicio del periodo.
            end_date (Optional[date], optional): Fecha final del periodo.
                Si no se proporciona, se toma `start_date + 7 días`.

        Returns:
            list[dict]: Lista de diccionarios con la siguiente información:
                - "sensor_id" (int): Identificador del sensor.
                - "date" (str): Fecha en formato ISO (YYYY-MM-DD).
                - "failure_seconds" (float | None): Duración del fallo en segundos.
        """
        end_date = start_date + timedelta(7) if end_date is None else end_date
        searched_week: list[date] = [
            (start_date + timedelta(days=i)) for i in range((end_date - start_date).days)
        ]
        return [
            {
                "sensor_id": sensor_id,
                "date": str(date),
                "failure_seconds": self.get_sensor_failure_duration(
                    sensor_id=sensor_id, searched_date=date
                ),
            }
            for date in searched_week
        ]

    def get_weekly_sensor_failure_probability(self, sensor_id: int, start_date: date) -> list[dict]:
        """Calcula la probabilidad diaria de fallo de un sensor durante una semana.

        La probabilidad se calcula como la proporción del tiempo en fallo
        respecto al total de segundos en una semana.

        Args:
            sensor_id (int): Identificador del sensor.
            start_date (date): Fecha de inicio del periodo.

        Returns:
            list[dict]: Lista de diccionarios con la siguiente información:
                - "sensor_id" (int): Identificador del sensor.
                - "date" (str): Fecha en formato ISO (YYYY-MM-DD).
                - "failure_probability" (str): Probabilidad del fallo (0 a 1) con 4 decimales.
        """
        weekly_sensor_failure_duration = self.get_weekly_sensor_failure_duration(
            sensor_id, start_date
        )
        return [
            {
                "sensor_id": sensor_id,
                "date": x["date"],
                "failure_probability": f"{x['failure_seconds'] / self.WEEK_TOTAL_SECONDS:.4f}",
            }
            for x in weekly_sensor_failure_duration
        ]

    def get_conditional_failure_probability(
        self,
        sensor_a_id: int,
        sensor_b_id: int,
        start_date: date,
        failure_minutes_threshold: float | None = 3,
    ) -> float:
        """Calcula la probabilidad condicional de que un sensor falle dado que otro sensor falló.

        Se considera que un sensor falló en un día si su duración de fallo
        es mayor o igual a un umbral en minutos.

        Fórmula:
            P(A|B) = (Número de días donde A y B fallaron) / (Número de días donde B falló)

        Args:
            sensor_a_id (int): Identificador del sensor A (sensor condicionado).
            sensor_b_id (int): Identificador del sensor B (sensor condicionante).
            start_date (date): Fecha de inicio del periodo.
            failure_minutes_threshold (Optional[float], optional): Umbral de fallo en minutos.
                Defaults a 3 minutos.

        Returns:
            float: Probabilidad condicional P(A|B) como un valor entre 0 y 1.
            Devuelve 0 si B nunca falló en el periodo.
        """
        failure_seconds_threshold = failure_minutes_threshold * 60

        sensor_b_weekly_failures = self.get_weekly_sensor_failure_duration(sensor_b_id, start_date)
        count_sensor_b_failures = sum(
            1
            for x in sensor_b_weekly_failures
            if x["failure_seconds"] and x["failure_seconds"] >= failure_seconds_threshold
        )

        if count_sensor_b_failures == 0:
            return 0.0

        sensor_a_weekly_failures = self.get_weekly_sensor_failure_duration(sensor_a_id, start_date)
        count_sensor_a_b_failures = sum(
            1
            for x, y in zip(sensor_a_weekly_failures, sensor_b_weekly_failures, strict=False)
            if x["failure_seconds"]
            and y["failure_seconds"]
            and x["failure_seconds"] >= failure_seconds_threshold
            and y["failure_seconds"] >= failure_seconds_threshold
        )

        return count_sensor_a_b_failures / count_sensor_b_failures

In [4]:
from datetime import datetime

searched_date = datetime.strptime("2025-08-01", "%Y-%m-%d").date()

extraction = LogExtractor(r"../data/logs.json")
analyzer = SensorFailureAnalyzer(logs=extraction.from_file())

In [5]:
analyzer.get_weekly_sensor_failure_probability(sensor_id="hidra_flow", start_date=searched_date)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

# Tu turno!
Ahora es tu turno! Ayudemos a nuestro equipo a solucionar los problemas detectados. 

1. Un fallo detectado al calcular el riesgo semanal de falla:
    - Al cacular el riesgo semanal de falla, dado el caso en que no exista tiempo de falla de un sensor para una fecha dentro del rango de fechas, la funcion falla.

        ```python
        searched_date = datetime.strptime("2025-08-01", "%Y-%m-%d").date()
        extraction = LogExtractor(
            r"../data/logs.json"
        )
        analyzer = SensorFailureAnalyzer(logs=extraction.from_file())

        analyzer.get_weekly_sensor_failure_probability(sensor_id="hidra_flow", start_date=searched_date)
        > TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'
        ```

- Modifica la funcion get_weekly_sensor_failure_probability 

---

2. Determina si el sensor buscado es valido:
    - Si el sensor buscado no es valido -- no es "oxy_guard", "thermo_core", "hidra_flow" o "radi_shield" --, se realizan todos los calculos, lo cual resulta en una perdida de computo. Dado que el sensor no exista, se deberia de fallar en lugar de realizar todos los calculos. 

        ```python
        searched_date = datetime.strptime("2025-08-01", "%Y-%m-%d").date()
        extraction = LogExtractor(
            r"../data/logs.json"
        )
        analyzer = SensorFailureAnalyzer(logs=extraction.from_file())

        analyzer.get_weekly_sensor_failure_duration(sensor_id="blalu", start_date=searched_date)
        > [{'sensor_id': 'blalu', 'date': '2025-08-01', 'failure_seconds': None},
        {'sensor_id': 'blalu', 'date': '2025-08-02', 'failure_seconds': None},
        {'sensor_id': 'blalu', 'date': '2025-08-03', 'failure_seconds': None},
        {'sensor_id': 'blalu', 'date': '2025-08-04', 'failure_seconds': None},
        {'sensor_id': 'blalu', 'date': '2025-08-05', 'failure_seconds': None},
        {'sensor_id': 'blalu', 'date': '2025-08-06', 'failure_seconds': None},
        {'sensor_id': 'blalu', 'date': '2025-08-07', 'failure_seconds': None}]
        ```

- Modifica las funciones fetch_logs_on_date y get_sensor_failure_duration para que lancen una excepcion si e sensor ingresado no es valido.

In [6]:
from datetime import date


class SensorFailureAnalyzer:
    """Clase para analizar fallos de sensores a partir de registros de eventos.

    Esta clase permite calcular la duración de fallos de sensores,
    la probabilidad de fallo semanal y la probabilidad condicional
    de que un sensor falle dado que otro sensor también falló.
    """

    WEEK_TOTAL_SECONDS: int = 7 * 24 * (60**2)
    """int: Número total de segundos en una semana."""

    def __init__(self, logs: list[Log]):
        """Inicializa el analizador con los registros de logs.

        Args:
            logs (list[Log]): Lista de objetos `Log` que contienen los eventos de los sensores.
        """
        self.logs = logs

    def get_sensor_failure_duration(self, sensor_id: str, searched_date: date) -> float | None:
        """Obtiene la duración total del fallo de un sensor en una fecha específica.

        Busca en los logs los eventos `FAILURE_START` y `FAILURE_END`
        del sensor en la fecha indicada y calcula el tiempo total en segundos.

        Args:
            sensor_id (str): Identificador del sensor a analizar.
            searched_date (date): Fecha para la que se quiere obtener la duración del fallo.

        Returns:
            float | None: Duración del fallo en segundos.
            Devuelve `None` si no hay logs del sensor en esa fecha.
        """
        logs_on_date: list[Log] = LogExtractor.fetch_logs_on_date(
            sensor_id=sensor_id, target_date=searched_date, logs=self.logs
        )

        if not logs_on_date:
            return None

        failure_start_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_START"), 0
        )
        failure_end_time: float = next(
            (log.duration_seconds for log in logs_on_date if log.event_type == "FAILURE_END"), 0
        )

        return abs(failure_end_time - failure_start_time)

    def get_weekly_sensor_failure_duration(
        self, sensor_id: str, start_date: date, end_date: date | None = None
    ) -> list[dict]:
        """Obtiene la duración diaria de fallos de un sensor durante una semana.

        Args:
            sensor_id (str): Identificador del sensor.
            start_date (date): Fecha de inicio del periodo.
            end_date (Optional[date], optional): Fecha final del periodo.
                Si no se proporciona, se toma `start_date + 7 días`.

        Returns:
            list[dict]: Lista de diccionarios con la siguiente información:
                - "sensor_id" (int): Identificador del sensor.
                - "date" (str): Fecha en formato ISO (YYYY-MM-DD).
                - "failure_seconds" (float | None): Duración del fallo en segundos.
        """
        assert sensor_id in ["oxy_guard", "thermo_core", "hidra_flow", "radi_shield"], (
            "The given sensor_id is not valid"
        )  # forma de solucionar el problema de sensor no valido

        end_date = start_date + timedelta(7) if end_date is None else end_date
        searched_week: list[date] = [
            (start_date + timedelta(days=i)) for i in range((end_date - start_date).days)
        ]
        return [
            {
                "sensor_id": sensor_id,
                "date": str(date),
                "failure_seconds": self.get_sensor_failure_duration(
                    sensor_id=sensor_id, searched_date=date
                ),
            }
            for date in searched_week
        ]

    def get_weekly_sensor_failure_probability(self, sensor_id: str, start_date: date) -> list[dict]:
        """Calcula la probabilidad diaria de fallo de un sensor durante una semana.

        La probabilidad se calcula como la proporción del tiempo en fallo
        respecto al total de segundos en una semana.

        Args:
            sensor_id (str): Identificador del sensor.
            start_date (date): Fecha de inicio del periodo.

        Returns:
            list[dict]: Lista de diccionarios con la siguiente información:
                - "sensor_id" (int): Identificador del sensor.
                - "date" (str): Fecha en formato ISO (YYYY-MM-DD).
                - "failure_probability" (str): Probabilidad del fallo (0 a 1) con 4 decimales.
        """
        weekly_sensor_failure_duration = self.get_weekly_sensor_failure_duration(
            sensor_id, start_date
        )
        return [
            {
                "sensor_id": sensor_id,
                "date": x["date"],
                "failure_probability": f"{x['failure_seconds'] / self.WEEK_TOTAL_SECONDS if x['failure_seconds'] is not None else None}",  # sln al problema de la division
            }
            for x in weekly_sensor_failure_duration
        ]

    def get_conditional_failure_probability(
        self,
        sensor_a_id: str,
        sensor_b_id: str,
        start_date: date,
        failure_minutes_threshold: float | None = 3,
    ) -> float:
        """Calcula la probabilidad condicional de que un sensor falle dado que otro sensor falló.

        Se considera que un sensor falló en un día si su duración de fallo
        es mayor o igual a un umbral en minutos.

        Fórmula:
            P(A|B) = (Número de días donde A y B fallaron) / (Número de días donde B falló)

        Args:
            sensor_a_id (str): Identificador del sensor A (sensor condicionado).
            sensor_b_id (str): Identificador del sensor B (sensor condicionante).
            start_date (date): Fecha de inicio del periodo.
            failure_minutes_threshold (Optional[float], optional): Umbral de fallo en minutos.
                Defaults a 3 minutos.

        Returns:
            float: Probabilidad condicional P(A|B) como un valor entre 0 y 1.
            Devuelve 0 si B nunca falló en el periodo.
        """
        failure_seconds_threshold = failure_minutes_threshold * 60

        sensor_b_weekly_failures = self.get_weekly_sensor_failure_duration(sensor_b_id, start_date)
        count_sensor_b_failures = sum(
            1
            for x in sensor_b_weekly_failures
            if x["failure_seconds"] and x["failure_seconds"] >= failure_seconds_threshold
        )

        if count_sensor_b_failures == 0:
            return 0.0

        sensor_a_weekly_failures = self.get_weekly_sensor_failure_duration(sensor_a_id, start_date)
        count_sensor_a_b_failures = sum(
            1
            for x, y in zip(sensor_a_weekly_failures, sensor_b_weekly_failures, strict=False)
            if x["failure_seconds"]
            and y["failure_seconds"]
            and x["failure_seconds"] >= failure_seconds_threshold
            and y["failure_seconds"] >= failure_seconds_threshold
        )

        return count_sensor_a_b_failures / count_sensor_b_failures

In [7]:
from datetime import datetime

searched_date = datetime.strptime("2025-08-01", "%Y-%m-%d").date()

extraction = LogExtractor(r"../data/logs.json")
analyzer = SensorFailureAnalyzer(logs=extraction.from_file())

In [8]:
analyzer.get_weekly_sensor_failure_probability(sensor_id="hidra_flow", start_date=searched_date)

[{'sensor_id': 'hidra_flow',
  'date': '2025-08-01',
  'failure_probability': '0.0005142195767195767'},
 {'sensor_id': 'hidra_flow',
  'date': '2025-08-02',
  'failure_probability': '0.0007705026455026455'},
 {'sensor_id': 'hidra_flow',
  'date': '2025-08-03',
  'failure_probability': '0.0007060185185185185'},
 {'sensor_id': 'hidra_flow',
  'date': '2025-08-04',
  'failure_probability': 'None'},
 {'sensor_id': 'hidra_flow',
  'date': '2025-08-05',
  'failure_probability': '0.0004464285714285714'},
 {'sensor_id': 'hidra_flow',
  'date': '2025-08-06',
  'failure_probability': '0.00035714285714285714'},
 {'sensor_id': 'hidra_flow',
  'date': '2025-08-07',
  'failure_probability': '0.0007506613756613756'}]

In [9]:
analyzer.get_weekly_sensor_failure_duration(sensor_id="blalu", start_date=searched_date)

AssertionError: The given sensor_id is not valid

3. Pruebas unitarias
    - Tu equipo no quiere que al modificar posteriormente alguna funcion, se genere un fallo en la funcion modificada o en otra. Por ello te han pedido que generes pruebas unitarias a tu codigo; asi mismo, te recomiendan usar el modulo `pytest`.

- Genera las pruebas unitarias correspondientes.


In [12]:
from datetime import datetime
from unittest.mock import patch

import pytest


def test_get_sensor_failure_duration_with_logs():
    FAILURE_START_DURATION = 419
    FAILURE_END_DURATION = 191
    EXPECTED_DURATION = abs(FAILURE_END_DURATION - FAILURE_START_DURATION)

    logs = [
        Log(
            sensor_id="radi_shield",
            event_type="FAILURE_START",
            timestamp=datetime.strptime("2025-08-11 13:08:35", "%Y-%m-%d %H:%M:%S"),
            duration_seconds=FAILURE_START_DURATION,
        ),
        Log(
            sensor_id="thermo_core",
            event_type="FAILURE_END",
            timestamp=datetime.strptime("2025-08-11 16:48:14", "%Y-%m-%d %H:%M:%S"),
            duration_seconds=FAILURE_END_DURATION,
        ),
    ]
    with patch("__main__.LogExtractor.fetch_logs_on_date", return_value=logs):
        analyzer = SensorFailureAnalyzer(logs)
        duration = analyzer.get_sensor_failure_duration("oxy_guard", date.today())
        assert duration == EXPECTED_DURATION


def test_get_sensor_failure_duration_no_logs():
    EXPECTED_DURATION = None

    with patch("__main__.LogExtractor.fetch_logs_on_date", return_value=[]):
        analyzer = SensorFailureAnalyzer([])
        duration = analyzer.get_sensor_failure_duration("oxy_guard", date.today())
        assert duration is EXPECTED_DURATION


def test_get_weekly_sensor_failure_duration_valid_sensor():
    DAYS_IN_WEEK = 7
    FAILURE_DURATION = 120

    logs = []
    analyzer = SensorFailureAnalyzer(logs)
    with patch.object(analyzer, "get_sensor_failure_duration", return_value=FAILURE_DURATION):
        start = date(2025, 1, 1)
        results = analyzer.get_weekly_sensor_failure_duration("oxy_guard", start)
        assert len(results) == DAYS_IN_WEEK
        assert all(r["sensor_id"] == "oxy_guard" for r in results)
        assert all(r["failure_seconds"] == FAILURE_DURATION for r in results)


def test_get_weekly_sensor_failure_duration_invalid_sensor():
    analyzer = SensorFailureAnalyzer([])
    with pytest.raises(AssertionError):
        analyzer.get_weekly_sensor_failure_duration("invalid_sensor", date.today())


def test_get_weekly_sensor_failure_probability():
    FAILURE_DURATION = 70
    EXPECTED_PROB = f"{FAILURE_DURATION / SensorFailureAnalyzer.WEEK_TOTAL_SECONDS}"

    analyzer = SensorFailureAnalyzer([])
    with patch.object(
        analyzer,
        "get_weekly_sensor_failure_duration",
        return_value=[
            {"date": "2025-01-01", "failure_seconds": FAILURE_DURATION, "sensor_id": "oxy_guard"},
            {"date": "2025-01-02", "failure_seconds": None, "sensor_id": "oxy_guard"},
        ],
    ):
        results = analyzer.get_weekly_sensor_failure_probability("oxy_guard", date(2025, 1, 1))
        assert results[0]["failure_probability"] == EXPECTED_PROB
        assert results[1]["failure_probability"] == "None"


def test_get_conditional_failure_probability():
    FAILURE_A = 200
    FAILURE_B = 180
    EXPECTED_PROB = 1.0

    analyzer = SensorFailureAnalyzer([])
    fake_week_a = [
        {"failure_seconds": FAILURE_A, "sensor_id": "oxy_guard", "date": "2025-01-01"},
        {"failure_seconds": 0, "sensor_id": "oxy_guard", "date": "2025-01-02"},
    ]
    fake_week_b = [
        {"failure_seconds": FAILURE_B, "sensor_id": "thermo_core", "date": "2025-01-01"},
        {"failure_seconds": 0, "sensor_id": "thermo_core", "date": "2025-01-02"},
    ]
    with patch.object(
        analyzer, "get_weekly_sensor_failure_duration", side_effect=[fake_week_a, fake_week_b]
    ):
        prob = analyzer.get_conditional_failure_probability(
            "oxy_guard", "thermo_core", date(2025, 1, 1), 3
        )
        assert prob == EXPECTED_PROB


def test_get_conditional_failure_probability_no_b_failures():
    EXPECTED_PROB = 0.0

    analyzer = SensorFailureAnalyzer([])
    fake_week_a = [{"failure_seconds": 100, "sensor_id": "oxy_guard", "date": "2025-01-01"}]
    fake_week_b = [{"failure_seconds": 0, "sensor_id": "thermo_core", "date": "2025-01-01"}]
    with patch.object(
        analyzer, "get_weekly_sensor_failure_duration", side_effect=[fake_week_a, fake_week_b]
    ):
        prob = analyzer.get_conditional_failure_probability(
            "oxy_guard", "thermo_core", date(2025, 1, 1), 3
        )
        assert prob == EXPECTED_PROB

In [13]:
import ipytest

ipytest.autoconfig()

ipytest.run()

.......                                                                                      [100%]
7 passed in 0.01s


<ExitCode.OK: 0>